In [2]:
import ccxt
import pandas as pd
from datetime import datetime as dt

In [5]:
exchange = ccxt.ftx({
    'apiKey': API_KEY,
    'secret': Secret_key,
})


In [4]:
API_KEY = 'DszcTlDUK6XdygwP-Ul8ZfEaQGru2rfdU1D4y3nZ'
Secret_key = 'UuiyB7uI5pVraqA9vLDWkQmPyY71MOoef-FfyKGU'


In [6]:
from_datetime = '2018-01-01 00:00:00'

from_timestamp = exchange.parse8601(from_datetime)

In [32]:
if exchange.has['fetchOrders']:
    since = exchange.milliseconds () - 86400000  # -1 day from now
    # alternatively, fetch from a certain starting datetime
    # since = exchange.parse8601('2018-01-01T00:00:00Z')
    all_orders = []
    while since < exchange.milliseconds ():
        symbol = 'btc/usd'  # change for your symbol
        limit = 4000  # change for your limit
        orders = await exchange.fetch_orders(symbol, since, limit)
        if len(orders):
            since = orders[len(orders) - 1]['timestamp']
            all_orders += orders
        else:
            break


TypeError: object list can't be used in 'await' expression


# Candles Data

In [27]:
ohlc = exchange.fetch_ohlcv('BTC/USDT', timeframe='1d', since = from_timestamp, limit = 1000)

In [28]:
df = pd.DataFrame(ohlc, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [432]:
df['Time']= df["Time"].apply(lambda x: dt.fromtimestamp(x/1000.0))

In [433]:
df['Time'] = df['Time'].dt.strftime('%Y-%m-%d')

In [452]:
df = df.set_index('Time')

KeyError: "None of ['Time'] are in the columns"

# Funding DATA

In [453]:

markets = exchange.load_markets()
# exchange.verbose = True  # uncomment for debugging

symbol = 'BTC/USDT'
market = exchange.market(symbol)

response = exchange.fapiPublic_get_fundingrate({
    'symbol': market['id'],
    'startTime': from_timestamp,  # ms to get funding rate from INCLUSIVE.
    # 'endTime': exchange.parse8601('2020-11-26T00:00:00Z'),  # ms to get funding rate until INCLUSIVE.
     'limit': 1000,  # default 100, max 1000
})


In [454]:
fundingdf = pd.DataFrame.from_dict(response)

In [455]:
fundingdf['fundingTime']= fundingdf['fundingTime'].astype(str).astype(int)

In [456]:
fundingdf = fundingdf.drop('symbol', axis = 1)

In [457]:
fundingdf['fundingTime']= fundingdf['fundingTime'].apply(lambda x: dt.fromtimestamp(x/1000.0))


In [458]:
fundingdf['fundingTime'] = fundingdf['fundingTime'].dt.strftime('%Y-%m-%d')

In [459]:
fundingdf = fundingdf.set_index('fundingTime')


In [464]:
df1 = pd.merge(df,fundingdf,left_index=True,right_index=True)

In [465]:
df1

,Open,High,Low,Close,Volume,fundingRate
